In [ ]:
from IPython.core.display import HTML
with open('../style.css') as f:
    css = f.read()
HTML(css)

# Alpha-Beta Pruning with proper Memoization

In order to have some variation in our game, we use random numbers to choose between optimal moves.
In order to have reproducible results, we use *seed* for the random number generator.

In [ ]:
import random
random.seed(0)

The global variable `Cache` is used as a cache for the function `value` defined later.  Given a `State`, the cache stores pairs of the form 
* `('=', v)`, 
* `('≤', v)`, or
* `('≥', v)`.

If `eval(State)` computes the value of a given `State`, then the following invariants are satisfied, provided 
`Cache[State]` is defined:
* $\texttt{Cache}[State] = (\texttt{'='}, v) \rightarrow \texttt{eval(State)} = v$.
* $\texttt{Cache}[State] = (\texttt{'≤'}, v) \rightarrow \texttt{eval(State)} \leq v$.
* $\texttt{Cache}[State] = (\texttt{'≥'}, v) \rightarrow \texttt{eval(State)} \geq v$.

In [ ]:
Cache = {}

Given a player `p`, the function `other(p)` computes the opponent of `p`.  This assumes that there are only two players and the set of all players is stored in the global variable `Players`.

In [ ]:
other = lambda p: [o for o in Players if o != p][0]

The variable `num_value_call` is a global variable that keeps track how often the function `value`, which is defined below, has been invoked.

In [ ]:
num_value_call = 0

The function `value` takes four arguments:
- `State` is the current state of the game,
- `player` is a player,
- `alpha` is the worst result that can happen to `player`,
- `beta` is the best result that can happen to `player`.

The function `value` returns the *value* that the given `State` has for `player` if both players play their optimal game.  This value is an element from the set $\{-1, 0, 1\}$.  
- If `player` can force a win, the return value is `1`.
- If `player` can at best force a draw, the return value is `0`.
- If `player` might loose even when playing optimal, the return value is `-1`.

For reasons of efficiency, this function is *memoized* using the global variable `Cache`
If `Cache[State]` is defined, then the computation of `value(State, player, alpha, beta)`
proceeds according to the following case distinction:
1. If the stored value $v$ is exact, we can return this value:

   $$\texttt{Cache[State]} = (\texttt{'='}, v) \rightarrow \texttt{value}(\texttt{State}, \alpha, \beta) = v.$$
2. If the stored value $v$ is an upper bound and this upper bound is less or equal than $\alpha$, then we know that
   the true value is less or equal than $\alpha$ and hence we can also return the value $v$:

   $$\texttt{Cache[State]} = (\texttt{'≤'}, v) \wedge v \leq \alpha \rightarrow 
     \texttt{value}(\texttt{State}, \alpha, \beta) = v.$$
3. If the stored value $v$ is an upper bound and this upper bound is bigger than $\alpha$ but less than $\beta$,   
   then we know that the true value is less or equal than $v$ and hence we shrink the interval $[\alpha, \beta]$
   into the interval $[\alpha, v]$.

   $$\texttt{Cache[State]} = (\texttt{'≤'}, v) \wedge \alpha < v < \beta \rightarrow 
     \texttt{value}(\texttt{State}, \alpha, \beta) = \texttt{alphaBeta}(\texttt{State}, \alpha, v).$$
     
   In this case the `Cache` is updated.  

4. If the stored value $v$ is an upper bound and this upper bound is bigger or equal than $\beta$,   
   then the stored value isn't of any help.
   
   $$\texttt{Cache[State]} = (\texttt{'≤'}, v) \wedge \beta \leq v \rightarrow 
     \texttt{value}(\texttt{State}, \alpha, \beta) = \texttt{alphaBeta}(\texttt{State}, \alpha, \beta).$$

   In this case, the cache needs to be updated.

5. If the stored value $v$ is a lower bound and this lower bound is greater or equal than $\beta$, then we 
   know that the true value is bigger or equal than $\beta$ and hence we can return the value $v$: 

   $$\texttt{Cache[State]} = (\texttt{'≥'}, v) \wedge \beta \leq v \rightarrow 
     \texttt{value}(\texttt{State}, \alpha, \beta) = v.$$
     
6. If the stored value $v$ is a lower bound and this lower bound is less than $\beta$ but bigger than $\alpha$,
   then we know that the true value is bigger or equal than $v$ and hence we shrink the interval $[\alpha,\beta]$
   into the interval $[v, \beta]$: 

   $$\texttt{Cache[State]} = (\texttt{'≥'}, v) \wedge \alpha < v < \beta \rightarrow 
     \texttt{value}(\texttt{State}, \alpha, \beta) = \texttt{alphaBeta}(\texttt{State}, v, \beta).$$
     
7. If the stored value $v$ is a lower bound and this lower bound is less or equal than $\alpha$,   
   then the stored value isn't of any help.
   
   $$\texttt{Cache[State]} = (\texttt{'≤'}, v) \wedge v \leq \alpha \rightarrow 
     \texttt{value}(\texttt{State}, \alpha, \beta) = \texttt{alphaBeta}(\texttt{State}, \alpha, \beta).$$

   In this case, the cache needs to be updated.

In [ ]:
def value(State, player, alpha=-1, beta=1):
    global Cache, num_value_call
    num_value_call += 1
    if State in Cache:
        flag, v = Cache[State]
        if flag == '=':
            return v
        if flag == '≤':
            if v <= alpha:
                return v
            if v < beta:
                w = alphaBeta(State, player, alpha, v)
                store_cache(State, alpha, v, w)
                return w
            if beta <= v:
                w = alphaBeta(State, player, alpha, beta)
                store_cache(State, alpha, beta, w)
                return w
        if flag == '≥':
            if beta <= v:
                return v
            if alpha < v:
                w = alphaBeta(State, player, v, beta)
                store_cache(State, v, beta, w)
                return w
            if v <= alpha:
                w = alphaBeta(State, player, alpha, beta)
                store_cache(State, alpha, beta, w)
                return w
    else:
        v = alphaBeta(State, player, alpha, beta)
        store_cache(State, alpha, beta, v)
        return v

The function `store_cache` is called with four arguments:
* `State` is a state of the game,
* `alpha` is a number,
* `beta`  is a number, and
* `value` is a number such that:
  $$\texttt{alphaBeta(State, alpha, beta)} = \texttt{value}$$
  
The function stores the `value` in the dictionary `Cache` under the key `State`.
It also stores an indicator that is either `'≤'`, `'='`, or `'≥'`.  The value that is stored 
satisfies the following conditions:
* If `Cache[State] = ('≤', value)`, then `eval(State) ≤ value`. 
* If `Cache[State] = ('=', value)`, then `eval(State) = value`. 
* If `Cache[State] = ('≥', value)`, then `eval(State) ≥ value`. 

In [ ]:
def store_cache(State, alpha, beta, value):
    global Cache
    if value <= alpha:
        Cache[State] = ('≤', value)
    elif value < beta:
        Cache[State] = ('=', value)
    else:
        Cache[State] = ('≥', value)

The function `alphaBeta` is called with four arguments:
* `State` is a state of the game,
* `alpha` is a number, and
* `beta`  is a number,

The function `alphaBeta` satisfies the following specification:
- $\alpha \leq \texttt{eval}(s, p) \leq \beta \;\rightarrow\;\texttt{alphaBeta}(s, p, \alpha, \beta) = \texttt{eval}(s,p)$
- $\texttt{eval}(s, p) < \alpha \;\rightarrow\; \texttt{alphaBeta}(s, p, \alpha, \beta) \leq \alpha$
- $\beta < \texttt{eval}(s, p) \;\rightarrow\; \beta \leq \texttt{alphaBeta}(s, p, \alpha, \beta)$

In [ ]:
def alphaBeta(State, player, alpha, beta):
    if finished(State):
        return utility(State, player)
    val = alpha
    for ns in next_states(State, player):
        val = max(val, -value(ns, other(player), -beta, -alpha))
        if val >= beta:
            return val
        alpha = max(val, alpha)
    return val

The function `best_move` takes two arguments:
- `State` is the current state of the game,
- `player` is a player.

The function `best_move` returns a pair of the form $(v, s)$ where $s$ is a state and $v$ is the value of this state.  The state $s$ is a state that is reached from `State` if `player` makes one of her optimal moves.  In order to have some variation in the game, the function randomly chooses any of the optimal moves.

In [ ]:
def best_move(State, player):
    NS        = next_states(State, player)
    bestVal   = value(State, player)
    BestState = random.choice([s for s in NS if -value(s, other(player)) == bestVal])
    return bestVal, BestState

The next line is needed because we need the function `IPython.display.clear_output` to clear the output in a cell.

In [ ]:
import IPython.display 

The function `play_game` plays on the given `canvas`.  The game played is specified indirectly by specifying the following:
- `Start` is a global variable defining the start state of the game.
- `next_states` is a function such that $\texttt{next_states}(s, p)$ computes the set of all possible states that can be reached from state $s$ if player $p$ is next to move.
- `finished` is a function such that $\texttt{finished}(s)$ is true for a state $s$ if the game is over in state $s$.
- `utility` is a function such that $\texttt{utility}(s, p)$ returns either `-1`, `0`, or `1` in the *terminal state* $s$.  We have that
  - $\texttt{utility}(s, p)= -1$ iff the game is lost for player $p$ in state $s$, 
  - $\texttt{utility}(s, p)=  0$ iff the game is drawn, and 
  - $\texttt{utility}(s, p)=  1$ iff the game is won for player $p$ in state $s$.

In [ ]:
def play_game(canvas):
    State = Start
    while (True):
        firstPlayer = Players[0]
        val, State  = best_move(State, firstPlayer);
        draw(State, canvas, f'For me, the game has the value {val}.')
        if finished(State):
            final_msg(State)
            break
        IPython.display.clear_output(wait=True)
        State = get_move(State)
        if finished(State):
            draw(State, canvas, '')
            IPython.display.clear_output(wait=True)
            final_msg(State)
            break

In [ ]:
%run Tic-Tac-Toe-Bitboard.ipynb

$\alpha$-$\beta$ pruning with memoization takes takes 25 ms to compute the value of the `Start` state. 

In [ ]:
%%time
val = value(Start, 0)

Regarding the number of states that are inspected by the function `value`, we have the following results :

| Algorithm                | Memoization  | Number of Calls | Size of Cache |
|:------------------------ | ------------ | ---------------:| -------------:|
| Minimax                  | `False`      |      $549\,946$ |        n.a.   |
| Minimax                  | `True`       |        $5\,478$ |     $5\,478$  |
| $\alpha$-$\beta$ pruning | `False`      |       $16\,811$ |        n.a.   |
| $\alpha$-$\beta$ pruning | `True`       |        $4\,321$ |     $2\,474$  |

In [ ]:
num_value_call

We check how many different states are stored in the `Cache`.  Without alpha-beta pruning, we had to inspect $5478$ different states, but now there are only
$2474$ different states in the cache.

In [ ]:
len(Cache)

Let's draw the board.

In [ ]:
Cache

In [ ]:
canvas = create_canvas()
draw(Start, canvas, f'Current value of game for "X": {val}')

Now its time to play.  In the input window that will pop up later, enter your move in the format "row,col"  with no space between row and column.

In [ ]:
play_game(canvas)